## 1. 线程的状态
- Java 中线程中状态可分为五种：New（新建状态），Runnable（就绪状态），Running（运行状态），Blocked（阻塞状态），Dead（死亡状态）
  - New：新建状态，当线程创建完成时为新建状态，即 new Thread(...)，还没有调用 start 方法时，线程处于新建状态
  - Runnable：就绪状态，当调用线程的的 start 方法后，线程进入就绪状态，等待 CPU 资源。处于就绪状态的线程由 Java 运行时系统的线程调度程序(thread scheduler)来调度
  - Running：运行状态，就绪状态的线程获取到 CPU 执行权以后进入运行状态，开始执行 run 方法
  - Blocked：阻塞状态，线程没有执行完，由于某种原因（如，I/O操作等）让出 CPU 执行权，自身进入阻塞状态
  - Dead：死亡状态，线程执行完成或者执行过程中出现异常，线程就会进入死亡状态

![image](page1.png)

---

## 2. wait/notify/notifyAll 方法的使用
### 2.1 wait 方法
- JDK 中一共提供了这三个版本的方法
  - wait() 方法的作用是将当前运行的线程挂起（即让其进入阻塞状态），直到 notify 或 notifyAll 方法来唤醒线程
  - wait(long timeout)，该方法与 wait() 方法类似，唯一的区别就是在指定时间内，如果没有 notify 或 notifAll 方法的唤醒，也会自动唤醒
  - wait(long timeout,long nanos)，本意在于更精确的控制调度时间，不过从目前版本来看(JDK 1.8)，该方法貌似没有完整的实现该功能
 
- demo：wait 方法的使用必须在同步的范围内，否则就会抛出 IllegalMonitorStateException 异常，wait 方法的作用就是阻塞当前线程等待 notify/notifyAll 方法的唤醒，或等待超时后自动唤醒

```
package com.paddx.test.concurrent;

public class WaitTest {
    // 增加 Synchronized 关键字
    public synchronized void testWait() {
        System.out.println("Start-----");
        try {
            wait(1000);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        System.out.println("End-------");
    }

    public static void main(String[] args) {
        final WaitTest test = new WaitTest();
        new Thread(new Runnable() {
            @Override
            public void run() {
                test.testWait();
            }
        }).start();
    }
}
```


### 2.2 notify/notifyAll 方法
- notify()：Wakes up a single thread that is waiting on this object's monitor.
- notifyAll()：Wakes up all threads that are waiting on this object's monitor.

- 只要在同一对象上去调用 notify/notifyAll 方法，就可以唤醒对应对象 monitor 上等待的线程了
- notify 和 notifyAll 的区别在于前者只能唤醒 monitor 上的一个线程，对其他线程没有影响，而 notifyAll 则唤醒所有的线程

```
package com.paddx.test.concurrent;

public class NotifyTest {
    public synchronized void testWait() {
        System.out.println(Thread.currentThread().getName() +" Start-----");
        try {
            wait(0);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        System.out.println(Thread.currentThread().getName() +" End-------");
    }

    public static void main(String[] args) throws InterruptedException {
        final NotifyTest test = new NotifyTest();
        for(int i = 0; i < 5; i++) {
            new Thread(new Runnable() {
                @Override
                public void run() {
                    test.testWait();
                }
            }).start();
        }

        synchronized (test) {
            test.notify();
        }
        
        Thread.sleep(3000);
        System.out.println("-----------分割线-------------");
        
        synchronized (test) {
            test.notifyAll();
        }
    }
}
```

- 调用 notify 方法时只有线程 Thread-0 被唤醒，但是调用 notifyAll 时，所有的线程都被唤醒了
  - 输出结果如下：
    - Thread-0 Start-----
    - Thread-1 Start-----
    - Thread-2 Start-----
    - Thread-3 Start-----
    - Thread-4 Start-----
    - Thread-0 End-------
    - -----------分割线-------------
    - Thread-4 End-------
    - Thread-3 End-------
    - Thread-2 End-------
    - Thread-1 End-------
  
#### 注意点
- 调用 wait 方法后，线程是会释放对 monitor 对象的所有权的
- 一个通过 wait 方法阻塞的线程，必须同时满足以下两个条件才能被真正执行
  - 线程需要被唤醒（超时唤醒或调用 notify/notifyll）
  - 线程唤醒后需要竞争到锁（monitor）
  
---

## 3. sleep/yield/join 方法解析
- 这几个方法都位于 Thread 类中，而上面三个方法都位于 Object 类中

### 3.1 sleep
- sleep 方法的作用是让当前线程暂停指定的时间（毫秒）
- 需要注意的是其与 wait 方法的区别：最简单的区别是，wait 方法依赖于同步，而 sleep 方法可以直接调用。而更深层次的区别在于 sleep 方法只是暂时让出 CPU 的执行权，并不释放锁。而 wait 方法则需要释放锁。

```
package com.paddx.test.concurrent;

public class SleepTest {
    public synchronized void sleepMethod() {
        System.out.println("Sleep start-----");
        try {
            Thread.sleep(1000);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        System.out.println("Sleep end-----");
    }

    public synchronized void waitMethod() {
        System.out.println("Wait start-----");
        synchronized (this) {
            try {
                wait(1000);
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
        System.out.println("Wait end-----");
    }

    public static void main(String[] args) {
        final SleepTest test1 = new SleepTest();

        for(int i = 0; i < 3; i++) {
            new Thread(new Runnable() {
                @Override
                public void run() {
                    test1.sleepMethod();
                }
            }).start();
        }

        try {
            // 暂停十秒，等上面程序执行完成
            Thread.sleep(10000);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        System.out.println("-----分割线-----");

        final SleepTest test2 = new SleepTest();

        for(int i = 0; i < 3; i++) {
            new Thread(new Runnable() {
                @Override
                public void run() {
                    test2.waitMethod();
                }
            }).start();
        }
    }
}
```

- 测试结果：通过 sleep 方法实现的暂停，程序是顺序进入同步块的，只有当上一个线程执行完成的时候，下一个线程才能进入同步方法。而 wait 方法则不同，当调用 wait 方法后，当前线程会释放持有的 monitor 对象锁。线程被唤醒后，需要竞争锁，获取到锁之后再继续执行。
```
Sleep start-----
Sleep end-----
Sleep start-----
Sleep end-----
Sleep start-----
Sleep end-----
-----分割线-----
Wait start-----
Wait start-----
Wait start-----
Wait end-----
Wait end-----
Wait end-----
```

---

### 3.2 yield
- yield 方法的作用是暂停当前线程，以便其他线程有机会执行，不过不能指定暂停的时间，并且也不能保证当前线程马上停止。yield 方法只是将 Running 状态转变为 Runnable 状态。

```
package com.paddx.test.concurrent;

public class YieldTest implements Runnable {
    @Override
    public void run() {
        try {
            Thread.sleep(100);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        for(int i = 0; i < 5; i++) {
            System.out.println(Thread.currentThread().getName() + ": " + i);
            Thread.yield();
        }
    }

    public static void main(String[] args) {
        YieldTest runn = new YieldTest();
        Thread t1 = new Thread(runn,"FirstThread");
        Thread t2 = new Thread(runn,"SecondThread");

        t1.start();
        t2.start();
    }
}
```

- 输出结果：通过 yield 方法来实现两个线程的交替执行。不过请注意：这种交替并不一定能得到保证
```
FirstThread: 0
SecondThread: 0
FirstThread: 1
SecondThread: 1
FirstThread: 2
SecondThread: 2
FirstThread: 3
SecondThread: 3
FirstThread: 4
SecondThread: 4
```

- 源码中的注释说明（yield 方法）
  - 调度器可能会忽略该方法
  - 使用的时候要仔细分析和测试，确保能达到预期的效果
  - 很少有场景要用到该方法，主要使用的地方是调试和测试

---

### 3.3 join
- void join：Waits for this thread to die.
- void join(long millis)：Waits at most millis milliseconds for this thread to die.
- void join(long millis, int nanos)：Waits at most millis milliseconds plus nanos nanoseconds for this thread to die.

- join 方法的作用是父线程等待子线程执行完成后再执行，换句话说就是将异步执行的线程合并为同步的线程。JKD 提供的 3 个方法效果和 wait 方法一模一样
- 查看源码可以看出 join 方法就是通过 wait 方法来将线程的阻塞，如果 join 的线程还在执行，则将当前线程阻塞起来，直到 join 的线程执行完成，当前线程才能执行。有一点需要注意，这里的 join 只调用了 wait 方法，却没有对应的 notify 方法，原因是 Thread 的 start 方法中做了相应的处理，所以当 join 的线程执行完成以后，会自动唤醒主线程继续往下执行。

- 不使用 join

```
package com.paddx.test.concurrent;

public class JoinTest implements Runnable{
    @Override
    public void run() {
        try {
            System.out.println(Thread.currentThread().getName() + " start-----");
            Thread.sleep(1000);
            System.out.println(Thread.currentThread().getName() + " end------");
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    }

    public static void main(String[] args) {
        for (int i = 0; i < 5; i++) {
            Thread test = new Thread(new JoinTest());
            test.start();
        }

        System.out.println("Finished~~~");
    }
}

执行结果如下：
Thread-0 start-----
Thread-1 start-----
Thread-2 start-----
Thread-3 start-----
Finished~~~
Thread-4 start-----
Thread-2 end------
Thread-4 end------
Thread-1 end------
Thread-0 end------
Thread-3 end------
```

- 使用 join 方法

```
package com.paddx.test.concurrent;

public class JoinTest implements Runnable{
    @Override
    public void run() {
        try {
            System.out.println(Thread.currentThread().getName() + " start-----");
            Thread.sleep(1000);
            System.out.println(Thread.currentThread().getName() + " end------");
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    }

    public static void main(String[] args) {
        for (int i = 0; i < 5; i++) {
            Thread test = new Thread(new JoinTest());
            test.start();
            try {
                test.join(); // 调用 join 方法
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }

        System.out.println("Finished~~~");
    }
}

执行结果如下：
Thread-0 start-----
Thread-0 end------
Thread-1 start-----
Thread-1 end------
Thread-2 start-----
Thread-2 end------
Thread-3 start-----
Thread-3 end------
Thread-4 start-----
Thread-4 end------
Finished~~~
```

- 对比两段代码的执行结果很容易发现，在没有使用 join 方法之间，线程是并发执行的，而使用 join 方法后，所有线程是顺序执行的

---

## 4. 总结
- wait/notify/notifyAll 方法的作用是实现线程间的协作，那为什么这三个方法不是位于 Thread 类中，而是位于 Object 类中？位于 Object 中，也就相当于所有类都包含这三个方法（因为 Java 中所有的类都继承自 Object 类）。
- wait 等待其实是对象 monitor，由于 Java 中的每一个对象都有一个内置的 monitor 对象，自然所有的类都理应有 wait/notify 方法。

---